In [1]:
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle, TextFace, PhyloNode

import pandas as pd
import numpy as np

from Bio import SeqIO, Seq, AlignIO

/home/sak0914/anaconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/sak0914/anaconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


# Construct trees for:

### 1. P_unique_HeV = all unique phosphoprotein sequences, outgroup = HeV
### 2. P_no_stop_codons_HeV = one sequence less than P_unique_HeV because of premature stop codon. USE THIS TREE FOR HYPOTHESIS TESTING.
### 3. P_BGD_dedup_HeV = Bangladesh sequences only, outgroup = HeV
### 4. P_whitmer_BGD = Whitmer et al. isolates, use midpoint rooting.

In [2]:
P_seqs = [seq.id for seq in SeqIO.parse("seq_for_analysis/P_unique_HeV.fasta", "fasta")]
metadata = pd.read_csv("metadata.csv").query("ID in @P_seqs").reset_index(drop=True)

# M_clade = metadata.query("Country in ['Malaysia', 'Cambodia']").ID.values
# B_clade = metadata.query("Country in ['Bangladesh', 'Thailand']").ID.values
# I_clade = metadata.query("Country in ['India']").ID.values

In [3]:
metadata["country_color"] = metadata["Country"].map({"Malaysia": "purple",
                                                     "India": "darkgreen",
                                                     "Bangladesh": "blue",
                                                     "Thailand": "black",
                                                     "Australia": "orange",
                                                     "Cambodia": "gray"
                                                    })

# need a dictionary mapping every node name to a color
country_color_dict = dict(zip(metadata["ID"], metadata["country_color"]))

In [4]:
def node_layout(color_dict=None):
    
    def layout(node):
        
        #if color_nodes:
        if node.is_leaf():
            #print(country_col_dict[node.name])

            if color_dict is not None:
                if node.name in list(color_dict.keys()):
                    font_color = color_dict[node.name]
                else:
                    font_color = "black"
            else:
                font_color = "black"

            N = AttrFace("name", 
                         fgcolor=font_color, 
                         fsize=7)

            faces.add_face_to_node(N, node, 0, 
                                  #position='branch-top',
                                   position="aligned"
                                  )
                
            # set all nodes to be the same -- black, small node size
            nstyle = NodeStyle()
            nstyle["fgcolor"] = "black"
            nstyle["size"] = 2
            node.set_style(nstyle)
            
    return layout

In [5]:
def tree_with_color_swatches(tree_file, color_dict, include_countries=[], remove_outgroup=False, out_group=None):
    
    t = Tree(tree_file)
    t.ladderize()
    
    if remove_outgroup:
        if out_group is None:
            raise ValueError("No outgroup specified!")
        else:
            search = t.search_nodes(name=out_group)[0]
            removed_node = search.detach()
    else:
        if out_group is not None:
            t.set_outgroup(out_group)  
        
    nst1 = NodeStyle()
    nst1["bgcolor"] = "LightSteelBlue"
    nst1["fgcolor"] = "black"
    nst1["size"] = 0
    
    nst2 = NodeStyle()
    nst2["bgcolor"] = "Moccasin"
    nst2["fgcolor"] = "black"
    nst2["size"] = 0
    
    nst3 = NodeStyle()
    nst3["bgcolor"] = "DarkSeaGreen"
    nst3["fgcolor"] = "black"
    nst3["size"] = 0
    
    for node in t.traverse():
        if not node.is_leaf():
            nstyle = NodeStyle()
            nstyle["size"] = 0
            node.set_style(nstyle)

    if "Bangladesh" in include_countries:
        # Bangladesh
        n1 = t.get_common_ancestor('AY988601', 'JN808857', 'JN808864', 'MK575060', 'MK575061',
           'MK575062', 'MK575067', 'MK575068', 'MK673565', 'MK673566',
           'MK673567', 'MK673568', 'MK673570', 'MK673571', 'MK673572',
           'MK673576', 'MK673579', 'MK673581', 'MK673582', 'MK673583',
           'MK673584', 'MK673585', 'MK673587', 'MK673588', 'MK673589',
           'MK673591', 'MK673592', 'MW535746')
        n1.set_style(nst1)

    if "Malaysia" in include_countries:
        # Malaysia
        n2 = t.get_common_ancestor('AF212302', 'AF376747', 'AJ627196', 'FN869553', 'MK673559',
           'MK801755')
        n2.set_style(nst2)
    
    if "India" in include_countries:
        # India
        n3 = t.get_common_ancestor('MH396625', 'MH523642', 'MN549402', 'MN549403',
           'MN549404', 'MN549405', 'MN549406', 'MN549409', 'MN549410')
        n3.set_style(nst3)
    
    ts = TreeStyle()
    ts.layout_fn = node_layout(color_dict)
    ts.show_leaf_name = False
    ts.show_branch_support = True
    
    #ts.title.add_face(TextFace("Phosphoprotein Sequences, 45", fsize=10), column=0)

    return t, ts

In [ ]:
t, ts = tree_with_color_swatches("trees/P_no_stop_codons_iqtree.nwk", country_color_dict, 
                                 include_countries=["Bangladesh", "Malaysia", "India"], out_group=None)

t.render(w=3000, tree_style=ts,
         #file_name="trees/Figures/P_unique_HeV.png", 
         file_name="%%inline"
         )

In [ ]:
t, ts = tree_with_color_swatches("trees/P_unique_HeV_iqtree.nwk", country_color_dict, 
                                 include_countries=["Bangladesh", "Malaysia", "India"], out_group="JN255817")

t.render(w=3000, tree_style=ts,
         #file_name="trees/Figures/P_unique_HeV.png", 
         file_name="%%inline"
         )

In [ ]:
t, ts = tree_with_color_swatches("trees/P_no_stop_codons_HeV_iqtree.nwk", country_color_dict,
                                 include_countries=["Bangladesh", "Malaysia", "India"], out_group="JN255817")

t.render(w=3000, tree_style=ts,
         #file_name="trees/Figures/P_no_stop_codons.png", 
         file_name="%%inline"
         )

In [ ]:
t, ts = tree_with_color_swatches("trees/P_no_stop_codons_HeV_iqtree.nwk", country_color_dict,
                                 include_countries=["Bangladesh", "Malaysia", "India"], remove_outgroup=True, out_group="JN255817")

t.render(w=3000, tree_style=ts,
         #file_name="trees/Figures/P_no_stop_codons.png", 
         file_name="%%inline"
         )

# save to get a new version without the Hendra outgroup
# t.write(outfile="trees/P_no_stop_codons_iqtree.nwk")

# Bangladesh Sequences Only

In [ ]:
BGD_seqs = [seq.id for seq in SeqIO.parse("seq_for_analysis/P_whitmer_BGD_dedup.fasta", "fasta")]

B_metadata = pd.read_csv("metadata_all.csv").query("ID in @BGD_seqs").reset_index(drop=True)
B_metadata["clade_color"] = B_metadata["Clade"].map({"B-1": "blue", "B-2": "red"})
B_clade_color_dict = dict(zip(B_metadata["ID"], B_metadata["clade_color"]))

In [ ]:
def basic_tree(tree_file, color_dict, out_group=None):
    
    t = Tree(tree_file)
    t.ladderize()
    
    if out_group is not None:
        t.set_outgroup(out_group)
    
    ts = TreeStyle()
    ts.layout_fn = node_layout(color_dict)
    ts.show_leaf_name = False
    ts.show_branch_support = True
    #ts.title.add_face(TextFace("Phosphoprotein Sequences, 45", fsize=10), column=0)
    
    for node in t.traverse():
        if not node.is_leaf():
            nstyle = NodeStyle()
            nstyle["size"] = 0
            node.set_style(nstyle)

    return t, ts

In [ ]:
t, ts = basic_tree("trees/P_BGD_dedup_HeV_iqtree.nwk", country_color_dict, out_group="JN255817")
#ts.scale = 1000 # 120 pixels per branch length unit

t.render(w=1500, tree_style=ts,
         #file_name="trees/Figures/P_BDG.png", 
         file_name="%%inline"
         )

In [ ]:
t, ts = basic_tree("trees/P_whitmer_BGD_dedup_iqtree.nwk", B_clade_color_dict)
#ts.scale = 1000 # 120 pixels per branch length unit

R = t.get_midpoint_outgroup()
# and set it as tree outgroup
t.set_outgroup(R)

t.render(w=1500, tree_style=ts,
         #file_name="trees/Figures/P_whitmer.png", 
         file_name="%%inline"
         )

# Trees with Multiple Sequence Alignments

## Positive selection sites, plot AA

In [ ]:
# Performs a tree reconciliation analysis
t = PhyloNode(newick="trees/P_unique_HeV_iqtree.nwk", alignment="trees/P_no_stop_codons_positive_selection.fasta", alg_format='fasta')
t.ladderize()

ts = TreeStyle()
ts.show_leaf_name = False
ts.show_branch_support = True
        
t.render(w=1500, 
       tree_style=ts,
         #file_name="trees/Figures/test.png", 
         file_name="%%inline"
         )

## Positive selection sites, plot Codons

In [ ]:
t = Tree("trees/P_unique_iqtree_ann.nwk")
t.ladderize()

ts = TreeStyle()

for n in t.traverse():
    if not n.is_leaf():
        nodestyle = NodeStyle()
        nodestyle["size"] = 0
        n.set_style(nodestyle)
    else:
        nodestyle = NodeStyle()
        nodestyle["size"] = 2
        nodestyle["fgcolor"] = "black"
        n.set_style(nodestyle)
        
ts.show_branch_support = True

t.render(w=3000, tree_style=ts,
         #file_name="trees/Figures/P_unique_iqtree_ann.png", 
         file_name="%%inline"
         )

In [ ]:
def node_layout_aln(node):
            
    if node.is_leaf():
        # set all nodes to be the same -- black, small node size
        nstyle = NodeStyle()
        nstyle["fgcolor"] = "black"
        nstyle["size"] = 1
        node.set_style(nstyle)
    else:    
        nstyle = NodeStyle()
        nstyle["size"] = 0
        node.set_style(nstyle)


def tree_with_aln(tree_file, aln, color_dict, out_group=None):

    # Performs a tree reconciliation analysis
    t = PhyloTree(tree_file)
    t.link_to_alignment(aln)
    
    if out_group is not None:
        t.set_outgroup(out_group)
        
    ts = TreeStyle()
    ts.show_leaf_name = False
    ts.show_branch_support = True
    #ts.layout_fn = node_layout_aln
    
    return t, ts

In [ ]:
#t, ts = tree_with_aln("trees/P_no_stop_codons_iqtree.nwk", "trees/P_no_stop_codons_positive_selection.fasta", country_color_dict)
  
# Performs a tree reconciliation analysis
t = PhyloTree("trees/P_unique_iqtree_ann.nwk")
t.link_to_alignment("seq_for_analysis/P_pos_codons.fasta")
t.ladderize()

ts = TreeStyle()
ts.show_leaf_name = False
ts.show_branch_support = True

# nstyle = NodeStyle()
# nstyle["fgcolor"] = "black"
# nstyle["size"] = 2
    
# for n in t.traverse():
#     # if n.is_leaf():
#     #     n.set_style(nstyle)
#     # else:
#     nstyle_internal = NodeStyle()
#     nstyle_internal["size"] = 0
#     n.set_style(nstyle_internal)
        
t.render(w=3000, tree_style=ts,
         file_name="trees/Figures/P_unique_iqtree_codons.png", 
         #file_name="%%inline"
         )

In [ ]:
t = Tree("trees/P_unique_iqtree_ann.nwk")
t.ladderize()

ts = TreeStyle()

for n in t.traverse():
    if not n.is_leaf():
        nodestyle = NodeStyle()
        nodestyle["size"] = 0
        n.set_style(nodestyle)
    else:
        nodestyle = NodeStyle()
        nodestyle["size"] = 2
        nodestyle["fgcolor"] = "black"
        n.set_style(nodestyle)
        
ts.show_branch_support = True

t.render(w=3000, tree_style=ts,
         #file_name="trees/Figures/P_unique_iqtree_ann.png", 
         file_name="%%inline"
         )

In [ ]:
t = Tree( "((a,b),c);" )

# Basic tree style
ts = TreeStyle()
ts.show_leaf_name = True

# Draws nodes as small red spheres of diameter equal to 10 pixels
nstyle = NodeStyle()
nstyle["shape"] = "sphere"
nstyle["size"] = 10
nstyle["fgcolor"] = "darkred"

# Gray dashed branch lines
nstyle["hz_line_type"] = 1
nstyle["hz_line_color"] = "#cccccc"

# Applies the same static style to all nodes in the tree. Note that,
# if "nstyle" is modified, changes will affect to all nodes
for n in t.traverse():
    
    if not n.is_leaf():
        nstyle = NodeStyle()
        nstyle["size"] = 0
        n.set_style(nstyle)

t.show(tree_style=ts)

In [ ]:
t, ts = tree_with_aln("trees/P_no_stop_codons_iqtree.nwk", "trees/P_no_stop_codons_positive_selection.fasta", country_color_dict)

#ts.layout_fn = node_layout_aln()
            
t.render(w=1500, tree_style=ts,
         #file_name="trees/Figures/P_no_stop_codons_msa.png", 
         file_name="%%inline"
         )